https://open-meteo.com/en/docs/gfs-api#latitude=29.4241&longitude=-98.4936&timezone=America%2FChicago

In [6]:
!pip install openmeteo_requests
!pip install requests_cache
!pip install retry_requests

In [7]:
import openmeteo_requests

import requests_cache
import pandas as pd
from retry_requests import retry

In [8]:
# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = 3600)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

In [9]:
# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://api.open-meteo.com/v1/forecast"
params = {
	"latitude": 29.4241,
	"longitude": -98.4936,
	"hourly": "temperature_2m",
	"timezone": "America/Chicago",
	"models": "gfs_seamless"
}

In [10]:
responses = openmeteo.weather_api(url, params=params)

# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

Coordinates 29.411029815673828°N -98.48500061035156°E
Elevation 203.0 m asl
Timezone b'America/Chicago' b'CST'
Timezone difference to GMT+0 -21600 s


In [11]:
# Process hourly data. The order of variables needs to be the same as requested.
hourly = response.Hourly()
hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()

hourly_data = {"date": pd.date_range(
	start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
	end = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = hourly.Interval()),
	inclusive = "left"
)}

In [12]:
hourly_data["temperature_2m"] = hourly_temperature_2m

hourly_dataframe = pd.DataFrame(data = hourly_data)
print(hourly_dataframe)

                         date  temperature_2m
0   2024-12-17 06:00:00+00:00       19.250000
1   2024-12-17 07:00:00+00:00       19.250000
2   2024-12-17 08:00:00+00:00       20.000000
3   2024-12-17 09:00:00+00:00       20.150000
4   2024-12-17 10:00:00+00:00       20.350000
..                        ...             ...
163 2024-12-24 01:00:00+00:00       18.123501
164 2024-12-24 02:00:00+00:00       18.173500
165 2024-12-24 03:00:00+00:00       18.023500
166 2024-12-24 04:00:00+00:00       17.323502
167 2024-12-24 05:00:00+00:00       16.373501

[168 rows x 2 columns]
